# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.core import ScriptRunConfig 
import os

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'capstone_hyperdrive'
project_folder = './pipeline-project'

experiment=Experiment(ws, experiment_name)

In [3]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
# amlcompute_cluster_name = "auto-ml"
amlcompute_cluster_name = "udacity-capstone-hyperdrive"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS3_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
from azureml.core.dataset import Dataset
from sklearn.model_selection import train_test_split

key = 'breast-cancer-test'
if key in ws.datasets.keys():
    dataset = ws.datasets[key]
    print("Dataset found! Next step.")

else:
    print("Please register dataset")

Dataset found! Next step.


In [5]:
df = dataset.to_pandas_dataframe()
train, test = train_test_split(df, shuffle=True)
train.to_csv('train.csv',index = False)

datastore = ws.get_default_datastore()
datastore.upload_files(files = ['./train.csv'])

train = Dataset.Tabular.from_delimited_files([(datastore,'train.csv')])

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}
Uploading an estimated of 1 files
Target already exists. Skipping upload for train.csv
Uploaded 0 files


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [6]:
from azureml.core import Environment, ScriptRunConfig
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=2)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling( {
    '--n_estimator': choice(100, 200, 500, 800, 1000),
    '--min_samples_split': choice(2, 5, 10),
    '--max_features': choice('auto', 'sqrt', 'log2')
})

if "training" not in os.listdir():
    os.mkdir("./training")

# get registered environment
env = Environment.from_conda_specification(name="sklearn-env",file_path="./dependencies.yaml")

# set up script run configuration
estimator = ScriptRunConfig(
    source_directory='.',
    script='train.py',
    compute_target=compute_target,
    environment=env
)

hyperdrive_run_config = HyperDriveConfig(run_config = estimator,
                                     hyperparameter_sampling = param_sampling, 
                                     policy = early_termination_policy,
                                     primary_metric_name = 'Accuracy',
                                     primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs = 20,
                                     max_concurrent_runs = 5)

In [7]:
# # TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
# early_termination_policy = <your policy here>

# #TODO: Create the different params that you will be using during training
# param_sampling = <your params here>

# #TODO: Create your estimator and hyperdrive config
# estimator = <your estimator here>

# hyperdrive_run_config = <your config here>

In [9]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_2eb4db88-fa98-4c67-9d1f-d3e5259ca142
Web View: https://ml.azure.com/runs/HD_2eb4db88-fa98-4c67-9d1f-d3e5259ca142?wsid=/subscriptions/a24a24d5-8d87-4c8a-99b6-91ed2d2df51f/resourcegroups/aml-quickstarts-271377/workspaces/quick-starts-ws-271377&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2025-01-12T09:00:38.5849360Z][GENERATOR][DEBUG]Sampled 5 jobs from search space 
[2025-01-12T09:00:38.8037512Z][SCHEDULER][INFO]Scheduling job, id='HD_2eb4db88-fa98-4c67-9d1f-d3e5259ca142_0' 
[2025-01-12T09:00:38.9858197Z][SCHEDULER][INFO]Scheduling job, id='HD_2eb4db88-fa98-4c67-9d1f-d3e5259ca142_2' 
[2025-01-12T09:00:38.9877991Z][SCHEDULER][INFO]Scheduling job, id='HD_2eb4db88-fa98-4c67-9d1f-d3e5259ca142_1' 
[2025-01-12T09:00:38.9887964Z][SCHEDULER][INFO]Scheduling job, id='HD_2eb4db88-fa98-4c67-9d1f-d3e5259ca142_3' 
[2025-01-12T09:00:38.9899321Z][SCHEDULER][INFO]Scheduling job, id='HD_2eb4db88-fa98-4c67-9d1f-d3e5259ca142_4' 
[2025-01-12T09:00:39.0969905Z]

{'runId': 'HD_2eb4db88-fa98-4c67-9d1f-d3e5259ca142',
 'target': 'udacity-capstone-hyperdrive',
 'status': 'Completed',
 'startTimeUtc': '2025-01-12T09:00:36.99882Z',
 'endTimeUtc': '2025-01-12T09:06:10.749548Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'b59c1373-50ed-4f77-a6c5-8cdf5a59b73c',
  'user_agent': 'python/3.10.11 (Linux-5.15.0-1073-azure-x86_64-with-glibc2.31) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.57.0',
  'best_child_run_id': 'HD_2eb4db88-fa98-4c67-9d1f-d3e5259ca142_1',
  'score': '0.95906432748538',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_2eb4db88-fa98-4c67-9d1f-d3e5259ca142_1'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType': '

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [10]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [11]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details())

{'runId': 'HD_2eb4db88-fa98-4c67-9d1f-d3e5259ca142_1', 'target': 'udacity-capstone-hyperdrive', 'status': 'Completed', 'startTimeUtc': '2025-01-12T09:01:28.425216Z', 'endTimeUtc': '2025-01-12T09:01:46.056854Z', 'services': {}, 'properties': {'_azureml.ComputeTargetType': 'amlctrain', '_azureml.ClusterName': 'udacity-capstone-hyperdrive', 'ContentSnapshotId': 'b59c1373-50ed-4f77-a6c5-8cdf5a59b73c', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': [], 'outputDatasets': [], 'runDefinition': {'script': 'train.py', 'command': '', 'useAbsolutePath': False, 'arguments': ['--max_features', 'log2', '--min_samples_split', '5', '--n_estimator', '500'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'udacity-capstone-hyperdrive', 'dataReferences': {}, 'data': {}, 'outputData': {}, 'datacaches': [], 'jobName': None, 'maxRunDurationSeconds': 2592000, 'nodeCount': 1, 'instanceTyp

In [12]:
#TODO: Save the best model
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

n_estimators: 500
min_samples_split: 5
max_features: log2
Accuracy 0.9590643274853801


In [13]:
model = best_run.register_model(model_name='model_hyperdrive', model_path='outputs/model_hyperdrive.pkl')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [14]:
from azureml.core.webservice import AciWebservice

aci_config = AciWebservice.deploy_configuration(
    cpu_cores=1,
    memory_gb=1,
    description='HyperDriver - Predict Wisoconsin Breast Cancer diagnosis - Malignant(1) or Benign(0)',
    auth_enabled=True
)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [19]:
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.automl.core.shared import constants

model = Model(ws, 'model_hyperdrive')


myenv = best_run.get_environment()
entry_script = 'predict_hyperdrive.py'
#best_run.download_file('outputs/scoring_file_v_1_0_0.py', entry_script)
#best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'myenv.yml')
inference_config = InferenceConfig(entry_script=entry_script, environment=myenv)

service = Model.deploy(workspace=ws, 
                       name='hyperdrive-breastcancer', 
                       models=[model], 
                       inference_config=inference_config, 
                       deployment_config=aci_config)

service.wait_for_deployment(show_output=True)

azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2025-01-12 09:12:06+00:00 Creating Container Registry if not exists.
2025-01-12 09:12:08+00:00 Use the existing image.
2025-01-12 09:12:08+00:00 Generating deployment configuration.
2025-01-12 09:12:10+00:00 Submitting deployment to compute.
2025-01-12 09:12:17+00:00 Checking the status of deployment hyperdrive-breastcancer..
2025-01-12 09:13:44+00:00 Checking the status of inference endpoint hyperdrive-breastcancer.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [20]:
service.update(enable_app_insights=True)

TODO: In the cell below, print the logs of the web service and delete the service

In [21]:
print("State "+ service.state)
print("Key " + service.get_keys()[0])
print("Swagger URI " + service.swagger_uri)
print("Scoring URI " + service.scoring_uri)

State Healthy
Key EbfxVZt73yrJAZzRI98VFfhYI8bi0hTW
Swagger URI http://56dafed1-86ed-4ba0-9bed-26e2192f3d67.westus2.azurecontainer.io/swagger.json
Scoring URI http://56dafed1-86ed-4ba0-9bed-26e2192f3d67.westus2.azurecontainer.io/score


**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

